In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

meta_file = "/kaggle/input/abo-dataset/images/metadata/images.csv"
df = pd.read_csv(meta_file)

print(df)

In [ ]:
import pandas as pd

images_df = pd.read_csv('/kaggle/input/abo-dataset/images/metadata/images.csv')


In [ ]:
import glob
import json

json_files = glob.glob('/kaggle/input/listings-images/listings_*.json')
metadata_records = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            record = json.loads(line)
            # Match main_image_id
            if 'main_image_id' in record:
                metadata_records.append({
                    'image_id': record['main_image_id'],
                    'metadata': record
                })
            # Match each other_image_id
            if 'other_image_id' in record:
                for oid in record['other_image_id']:
                    metadata_records.append({
                        'image_id': oid,
                        'metadata': record
                    })

metadata_df = pd.DataFrame(metadata_records)


In [ ]:
metadata_df = metadata_df.drop_duplicates(subset=['image_id'])


In [ ]:
merged_df = images_df.merge(metadata_df, how='left', on='image_id')


In [ ]:
def extract_metadata_fields(meta):
    # Helper to get first value from a list of dicts, or None
    def get_first(meta, key, subkey='value'):
        try:
            if isinstance(meta.get(key), list) and meta[key]:
                return meta[key][0].get(subkey)
        except Exception:
            pass
        return None

    # Helper for bullet_point and item_keywords: join all values
    def get_all(meta, key):
        try:
            if isinstance(meta.get(key), list):
                return '^'.join([str(d.get('value')) for d in meta[key] if 'value' in d])
        except Exception:
            pass
        return None

    # Helper for node_name: join all node_names if multiple
    def get_node_names(meta):
        try:
            if isinstance(meta.get('node'), list):
                return '^'.join([str(d.get('node_name')) for d in meta['node'] if 'node_name' in d])
        except Exception:
            pass
        return None

    return pd.Series({
        'color': get_first(meta, 'color'),
        'product_type': get_first(meta, 'product_type'),
        'item_name': get_first(meta, 'item_name'),
        'bullet_point': get_all(meta, 'bullet_point'),
        'item_keywords': get_all(meta, 'item_keywords'),
        'node_name': get_node_names(meta)
    })


In [ ]:
extracted = merged_df['metadata'].apply(extract_metadata_fields)
final_df = pd.concat([merged_df, extracted], axis=1)


In [ ]:
print(final_df[['color', 'product_type', 'item_name', 'bullet_point', 'item_keywords', 'node_name']].head())


In [ ]:
changed_final_df = final_df.drop(columns=['metadata'])


In [ ]:
com_df = pd.read_csv('/kaggle/input/merged-one/merged_vqa_dataset_output.csv')
com_df